In [87]:
import pickle
import networkx as nx
import numpy as np
import math
import EoN
import random

In [88]:
path = '/home/platform/project/crustdb_platform/crustdb_api/workspace/crustdb_database/topology/Lung/Lung5_Rep1.gem.txt.cell_type.B-cell_ISV2/MST/MST.pkl'
with open(path, 'rb') as handle:
    G = pickle.load(handle)

In [89]:
# G.number_of_nodes()

In [90]:
G.nodes

NodeView(('MZT2A', 'MALAT1', 'DUSP5', 'CD74', 'TYK2', 'HLA-B', 'IGHM', 'HLA-C', 'B2M', 'IGHA1', 'HLA-DRB1', 'HLA-A', 'IGKC', 'HLA-DPA1', 'COL9A2', 'RPL21', 'IGHG1', 'HLA-DRA', 'CXCR4', 'RPL34', 'NPPC', 'NEAT1', 'CD37', 'RPL37', 'IGHG2', 'HLA-DRB5', 'HLA-E', 'CIITA', 'FKBP11', 'MS4A1', 'CD53', 'BTG1', 'HLA-DPB1', 'RPL22', 'TIMP1', 'IL16', 'HLA-DQA1', 'COL1A1', 'HSPA1A', 'HSP90AB1', 'PSAP', 'HSPA1B', 'IGFBP7', 'VIM', 'EZR', 'LTB', 'TNFRSF12A', 'HLA-DQB1', 'IGHD', 'CD79A', 'COL3A1', 'CD52', 'TUBB', 'PTPRC', 'HSP90AA1', 'IFITM3', 'MT2A', 'EIF5A', 'CD19', 'PTGDR2', 'CD3E', 'IRF3', 'KRT14', 'RPL32', 'SRGN', 'PTPRCAP', 'BGN', 'MIF', 'COTL1', 'KLF2', 'CD44', 'GPX1', 'S100A6', 'GSTP1', 'TNFRSF13B', 'IL32', 'BCL2', 'MGP', 'CD83', 'HSPB1', 'JAK1', 'HSP90B1', 'SOD2', 'JCHAIN', 'TAGLN', 'DUSP1', 'CD81', 'NLRC5', 'SELL', 'CCL19', 'CRP', 'CD63', 'VHL', 'CD48', 'RAC2', 'ZFP36', 'H4C3', 'P2RX5', 'JUNB', 'CTNNB1', 'CLU', 'SQSTM1', 'ARHGDIB', 'LGALS1', 'RAMP1', 'LYZ', 'CD27', 'WIF1', 'PTGDS', 'FN1', 'DCN

In [91]:
node_size = nx.pagerank(G, alpha=0.85,weight=None)
sorted_dict = sorted(node_size.items(), key=lambda x: x[1],reverse=True)
labels = {}
for i in sorted_dict[:10]:
    labels[i[0]] = i[0]
labels

{'CD37': 'CD37',
 'CRIP1': 'CRIP1',
 'WIF1': 'WIF1',
 'MMP2': 'MMP2',
 'FABP5': 'FABP5',
 'IFITM3': 'IFITM3',
 'CXCR4': 'CXCR4',
 'FYN': 'FYN',
 'JAK1': 'JAK1',
 'KRT14': 'KRT14'}

In [92]:
root = list(labels.values())[0]

In [93]:
def _hierarchy_pos(G, root, leftmost, width, leafdx = 0.2, vert_gap = 0.2, vert_loc = 0, 
                xcenter = 0.5, rootpos = None, 
                leafpos = None, parent = None):
    # pos: a dict saying where all nodes go if they have been assigned
    # parent: parent of this branch. - only affects it if non-directed

    if rootpos is None:
        rootpos = {root:(xcenter,vert_loc)}
    else:
        rootpos[root] = (xcenter, vert_loc)
    if leafpos is None:
        leafpos = {}
    children = list(G.neighbors(root))
    leaf_count = 0
    if not isinstance(G, nx.DiGraph) and parent is not None:
        children.remove(parent)  
    if len(children)!=0:
        rootdx = width/len(children)
        nextx = xcenter - width/2 - rootdx/2
        for child in children:
            nextx += rootdx
            rootpos, leafpos, newleaves = _hierarchy_pos(G,child, leftmost+leaf_count*leafdx, 
                                width=rootdx, leafdx=leafdx,
                                vert_gap = vert_gap, vert_loc = vert_loc-vert_gap, 
                                xcenter=nextx, rootpos=rootpos, leafpos=leafpos, parent = root)
            leaf_count += newleaves

        leftmostchild = min((x for x,y in [leafpos[child] for child in children]))
        rightmostchild = max((x for x,y in [leafpos[child] for child in children]))
        leafpos[root] = ((leftmostchild+rightmostchild)/2, vert_loc)
    else:
        leaf_count = 1
        leafpos[root]  = (leftmost, vert_loc)
#        pos[root] = (leftmost + (leaf_count-1)*dx/2., vert_loc)
#        print(leaf_count)
    return rootpos, leafpos, leaf_count

rootpos, leafpos, leaf_count = _hierarchy_pos(G, root, leftmost=0, width=1)

In [96]:
def my_hierarchy(G, root, parent = None):
    children = list(G.neighbors(root))
    if not isinstance(G, nx.DiGraph) and parent is not None:
        children.remove(parent)
    child_list = {}
    if len(children)!=0:
        for child in children:
            # if child == root: print('--------------')
            x = my_hierarchy(G, child, parent=root)
            child_list[child] = x
    return child_list

res = my_hierarchy(G, root)

In [8]:
with open('/home/platform/project/crustdb_platform/crustdb_api/workspace/analysis_script/tmp/my_cronjob.log', 'a') as f:
    f.write('exec update start  '+str(1)+"\n")
    f.write('tasklist'+"\n")

In [4]:
import subprocess
import sys

def get_job_status(job_id):
    squeue_command = ["squeue", "--job", str(job_id), "--format=%T"]
    try:
        squeue_output = subprocess.check_output(squeue_command).decode("utf-8")
        lines = squeue_output.strip().split("\n")
        if len(lines) > 1:
            return lines[1].strip()
    except subprocess.CalledProcessError as e:
        print("squeue check error:", e, file=sys.stderr)
        pass

    sacct_command = ["sacct", "--jobs", str(job_id), "--format=JobID,State"]
    try:
        print(sacct_command)
        sacct_output = subprocess.check_output(sacct_command).decode("utf-8")
    except subprocess.CalledProcessError as e:
        print("sacct check error:", e, file=sys.stderr)
        return None
    lines = sacct_output.strip().split("\n")
    for line in lines[1:]:
        parts = line.strip().split()
        if len(parts) == 2 and parts[0] == str(job_id):
            return parts[1]
    return None

In [6]:
s = get_job_status(1410439)

['sacct', '--jobs', '1410439', '--format=JobID,State']


slurm_load_jobs error: Invalid job id specified
squeue check error: Command '['squeue', '--job', '1410439', '--format=%T']' returned non-zero exit status 1.


In [7]:
s

'COMPLETED'

In [1]:
import re

In [2]:
with open('/home/platform/project/crustdb_platform/crustdb_api/workspace/analysis_script/user/1718914601_CLXV/output/result/SS200000108BR_A3A4_scgem.Endothelial_cell_NH17/SS200000108BR_A3A4_scgem.Endothelial_cell_NH17.log') as f:
    L = f.readlines()
str = ''
for i in L:
    str += i

In [3]:
str

"Adding SS200000108BR_A3A4_scgem.Endothelial_cell in adata.obs['orig.ident'].\nAdding data.position as adata.obsm['spatial'] .\nAdding data.position as adata.obs['x'] and adata.obs['y'] .\nFinished conversion to anndata.\nSpecies: Mice\nSample Name: SS200000108BR_A3A4_scgem.Endothelial_cell\nSeed: 276\nCell Number: 3742\nGene Number: 20620\nThreshold for gene filter is: 0.9\nNumber of genes used for Rotation Derivation is: 20\nTask ID: NH17\n\nRMSD between New Configuration and Old Configuration for loop 1 is: 1.5460099142863046\nRMSD between New Configuration and Old Configuration for loop 2 is: 1.1024348190870825\nRMSD between New Configuration and Old Configuration for loop 3 is: 0.7504339278753149\nRMSD between New Configuration and Old Configuration for loop 4 is: 0.5617881870985418\nRMSD between New Configuration and Old Configuration for loop 5 is: 0.5042391766914847\nRMSD between New Configuration and Old Configuration for loop 6 is: 0.47996502393980306\nRMSD between New Config

In [3]:
import re
def re_match(start, end, str):
    _, res = re.findall(r"("+start+r")\s*(.*?)\s*(?!\1)(?:"+end+r")", str)[0]
    return res
re_match('Species: ', '\n', str)

In [5]:
re.match(r"(?P<first_name>\w+) (?P<last_name>\w+)", "Malcom Reynolds").groupdict()

{'first_name': 'Malcom', 'last_name': 'Reynolds'}

In [8]:
import csv
cStr = '10.1038/s41467-023-43458-x,IRF7_DCs_UPBD,IRF7_DCs,human_breast_cancer_S1R2,Xenium,Homo sapiens (Human),N/A,"TNM stage T2N1M0, ER+ /HER2+ /PR-",N/A'
[ "{}".format(x) for x in list(csv.reader([cStr], delimiter=',', quotechar='"')) ]

["['10.1038/s41467-023-43458-x', 'IRF7_DCs_UPBD', 'IRF7_DCs', 'human_breast_cancer_S1R2', 'Xenium', 'Homo sapiens (Human)', 'N/A', 'TNM stage T2N1M0, ER+ /HER2+ /PR-', 'N/A']"]

In [11]:
list(csv.reader([cStr], delimiter=',', quotechar='"'))[0]

['10.1038/s41467-023-43458-x',
 'IRF7_DCs_UPBD',
 'IRF7_DCs',
 'human_breast_cancer_S1R2',
 'Xenium',
 'Homo sapiens (Human)',
 'N/A',
 'TNM stage T2N1M0, ER+ /HER2+ /PR-',
 'N/A']

In [2]:
import scanpy as sc

adata = sc.read_h5ad('/home/platform/project/crustdb_platform/crustdb_api/adata.h5ad')

In [7]:
adata.obsm['Rotation']

array([[[-0.78412063,  0.22081987, -0.57999432],
        [ 0.29400593, -0.69086012, -0.6605095 ],
        [-0.54654857, -0.6884409 ,  0.47679532]],

       [[ 0.27859316,  0.86439489,  0.41857775],
        [ 0.15997551, -0.47151002,  0.867229  ],
        [ 0.94699192, -0.17464188, -0.26964147]],

       [[-0.1516805 , -0.88806812,  0.43396778],
        [-0.93358874,  0.27292874,  0.23221103],
        [-0.32466149, -0.36992555, -0.87048837]],

       ...,

       [[-0.33130616,  0.52825495,  0.7817819 ],
        [ 0.9148131 , -0.02299753,  0.40322215],
        [ 0.23098315,  0.84877431, -0.47563533]],

       [[ 0.19530004,  0.3409872 ,  0.9195573 ],
        [ 0.77764714, -0.62515726,  0.06665827],
        [ 0.59759754,  0.70207274, -0.38726096]],

       [[-0.04208385,  0.05086813,  0.99781831],
        [-0.96775946,  0.24615808, -0.05336507],
        [-0.24833562, -0.96789392,  0.03886884]]])

In [3]:
from django.conf import global_settings
global_settings

<module 'django.conf.global_settings' from '/home/platform/.local/lib/python3.9/site-packages/django/conf/global_settings.py'>

In [5]:
! cat /home/platform/.local/lib/python3.9/site-packages/django/conf/global_settings.py | grep DATA_UPLOAD_MAX_MEMORY_SIZE

DATA_UPLOAD_MAX_MEMORY_SIZE = 2621440  # i.e. 2.5 MB


In [6]:
gem_path = '/home/platform/project/crustdb_platform/crustdb_web/public/demo_input/craft_multi_celltype/merfish_ileum/baysor_transcripts.gem.csv'
sep = ','

def read_gem_header(gem_path, sep="\t"):
    with open(gem_path, "r") as f:
        header = f.readline().strip().split(sep)
        return header

def check_gem_header(header):
    if (
        ("geneID" in header or "gene" in header or "GeneID" in header)
        and ("cell" in header or "CellID" in header or "cell_id" in header)
        and "x" in header
        and "y" in header
        and ("MIDCount" in header or "MIDCounts" in header or "expr" in header)
    ):
        return True
    else:
        return False

gem_header = read_gem_header(gem_path, sep=',')
check_gem_header(gem_header)

True

In [1]:
str = 'MERFISH_MICE_ILEUM.csv.leiden_final.B_Follicular_Circulating_FQ1Z'
str2 = str.replace('MERFISH_MICE_ILEUM.csv', 'transcripts.gem.csv')
str2

'transcripts.gem.csv.leiden_final.B_Follicular_Circulating_FQ1Z'

In [17]:
import pandas as pd
import numpy as np

In [18]:
map = {
    'S1R1': 'GSM7780153_Xenium_R1_Fig1-5_supervised.csv',
    'S1R2': 'GSM7780154_Xenium_R2_Fig1-5_supervised.csv',
    'S2': 'GSM7780155_Xenium_Fig_6_unsupervised.csv',
}

In [19]:
for i, j in map.items():
    print(i, j)
    df1 = pd.read_csv('/home/platform/project/crustdb_platform/crustdb_api/workspace/crustdb_database/adata/still_adata/'+i+'/'+j)
    df2 = pd.read_csv('/home/platform/project/crustdb_platform/crustdb_api/workspace/crustdb_database/adata/still_adata/'+i+'/cells.csv')
    df2['Cluster'] = df1['Cluster']
    df2.to_csv('/home/platform/project/crustdb_platform/crustdb_api/workspace/crustdb_database/adata/'+j)

S1R1 GSM7780153_Xenium_R1_Fig1-5_supervised.csv
S1R2 GSM7780154_Xenium_R2_Fig1-5_supervised.csv
S2 GSM7780155_Xenium_Fig_6_unsupervised.csv


In [2]:
import pandas as pd
import numpy as np

In [11]:
df = pd.read_csv('/home/platform/project/crustdb_platform/crustdb_api/workspace/crustdb_database/main/CRUST_OUTPUT_INFO.csv')
np.unique([i[:-5] for i in df['Data UID'].to_numpy()]).shape

(440,)

In [13]:
df.head()

,DOI,Data UID,Cell Type,Slice ID,ST Platform,Species,Developmental Stage / Age,Disease Stage,Sex
0,10.1126/science.abp9444,Stage44.CP_1XOH,CP,Stage44_telencephalon_rep2_FP200000239BL_E4,Stereo-seq,Ambystoma mexicanum (Axolotl),Stage 44,Normal,Male
1,10.1126/science.abp9444,Stage44.dEGC_7T6P,dEGC,Stage44_telencephalon_rep2_FP200000239BL_E4,Stereo-seq,Ambystoma mexicanum (Axolotl),Stage 44,Normal,Male
2,10.1126/science.abp9444,Stage44.dNBL1_HM1K,dNBL1,Stage44_telencephalon_rep2_FP200000239BL_E4,Stereo-seq,Ambystoma mexicanum (Axolotl),Stage 44,Normal,Male
3,10.1126/science.abp9444,Stage44.dNBL2_FUQU,dNBL2,Stage44_telencephalon_rep2_FP200000239BL_E4,Stereo-seq,Ambystoma mexicanum (Axolotl),Stage 44,Normal,Male
4,10.1126/science.abp9444,Stage44.dNBL3_HF1K,dNBL3,Stage44_telencephalon_rep2_FP200000239BL_E4,Stereo-seq,Ambystoma mexicanum (Axolotl),Stage 44,Normal,Male


In [22]:
([i[:-5] for i in df[df['Species'] == 'Ambystoma mexicanum (Axolotl)']['Data UID']])

array(['Adult.CMPN', 'Adult.CP', 'Adult.MSN', 'Adult.VLMC', 'Adult.cckIN',
       'Adult.dpEX', 'Adult.mpEX', 'Adult.mpIN', 'Adult.nptxEX',
       'Adult.ntng1IN', 'Adult.ribEGC', 'Adult.scgnIN', 'Adult.sfrpEGC',
       'Adult.sstIN', 'Adult.tlNBL', 'Adult.wntEGC', 'Control_Juv.CMPN',
       'Control_Juv.CP', 'Control_Juv.MCG', 'Control_Juv.MSN',
       'Control_Juv.Unknown', 'Control_Juv.VLMC', 'Control_Juv.cckIN',
       'Control_Juv.dpEX', 'Control_Juv.mpEX', 'Control_Juv.nptxEX',
       'Control_Juv.npyIN', 'Control_Juv.ribEGC', 'Control_Juv.scgnIN',
       'Control_Juv.sfrpEGC', 'Control_Juv.sstIN', 'Control_Juv.tlNBL',
       'Control_Juv.wntEGC', 'Meta.CMPN', 'Meta.CP', 'Meta.MCG',
       'Meta.MSN', 'Meta.Oligo', 'Meta.Unknown', 'Meta.VLMC',
       'Meta.cckIN', 'Meta.dpEX', 'Meta.mpEX', 'Meta.nptxEX',
       'Meta.npyIN', 'Meta.ribEGC', 'Meta.scgnIN', 'Meta.sfrpEGC',
       'Meta.sstIN', 'Meta.tlNBL', 'Meta.wntEGC', 'Stage44.CP',
       'Stage44.Immature_CMPN', 'Stage44.Immatu